In [11]:
import re

# COMMMENTS

# (1) : we assumed this pattern is valid : a*?  (https://regex101.com)
# (2) : ?? is assumed ? [ valid ]
# (3) : ++ is assumed + [ valid ]
# (4) : ** is invalid

open_list = ["[","("]
close_list = ["]",")"]
# Function to check parentheses
def check_braces(myStr):
    stack = []
    for i in myStr:
        if i in open_list:
            stack.append(i)
        elif i in close_list:
            pos = close_list.index(i)
            if ((len(stack) > 0) and
                (open_list[pos] == stack[len(stack)-1])):
                stack.pop()
            else:
                return False
    if len(stack) == 0:
        return True
    else:
        return False

def check_ORs(regex):
  ORs = regex.count('|')
  splitted = list(filter(lambda x: x, regex.split('|')))
  if(len(splitted) != ORs + 1):
    return False
  return True

def check_valid_question_mark(regex):
  matches = re.findall(r'\?{3,}', regex)
  if(len(matches) > 0):
    return False

  while(regex.find("??") != -1):
    regex = regex.replace("??", "?")

  if(regex[0] == '?'):
    return False

  if(regex.find('|?') != -1):
    return False

  return regex

def check_valid_plus(regex):
  matches = re.findall(r'\+{3,}', regex)
  if(len(matches) > 0):
    return False

  while(regex.find("++") != -1):
    regex = regex.replace("++", "+")

  if(regex[0] == '+'):
    return False

  if(regex.find('|+') != -1):
    return False

  return regex



def check_valid_star(regex):
  matches = re.findall(r'\*{2,}', regex)
  if(len(matches) > 0):
    return False

  if(regex[0] == '*'):
    return False

  if(regex.find('|*') != -1):
    return False

  return True


def validate_regex(regex):
  # check ORs (|)
  if (not check_ORs(regex)):
    return False

  # check braces
  if(not check_braces(regex)):
    return False

  # check ?
  regex = check_valid_question_mark(regex)
  if(not regex):
    return False

  # check +
  regex = check_valid_plus(regex)
  if(not regex):
    return False

  # check *
  if(not check_valid_star(regex)):
    return False

  pettern = "[a-zA-Z0-9\.\+\?\*\[\]\(\)\-\|]"
  matched = re.findall(pettern, regex)
  if(len(matched) != len(regex)):
    return False

  return True

In [12]:
def validateRegex(regex):
    try:
        re.compile(regex)
    except re.error:
        print(f"Invalid input: {regex}")
        return False
    return True

In [13]:
class postfixByShuntYard:
    def __init__(self, regex):
        self.regex = regex
        self.postfix = self.applyShuntYard(regex)

    def getPostfixedRegex(self):
        return self.postfix

    def applyShuntYard(self, regex):
        # *, +, ? : have the same precendence level
        # . : mid precendence level
        # | : lowest precedence level
        symbolPrecedence = {'*': 5, '+': 4, '?': 3, '.': 2, '|': 1}
        postfix = ""
        stack = []
        # If we find [xyz] as example, this would be converted to (x|y|z)
        for i in range(len(regex)):
            if regex[i] == '[':
                j = i + 1
                while regex[j] != ']':
                    if regex[j].isalnum() and regex[j + 1].isalnum():
                        regex = regex[:j + 1] + '|' + regex[j + 1:]
                    j += 1

        # Replace the [] with ()
        regex = regex.replace('[', '(')
        regex = regex.replace(']', ')')

        print("postfix1: ", regex)

        # If we find a-f would be converted to (a|b|c|d|e|f)
        for i in range(regex.count('-')):
            for j in range(len(regex)):
                character = regex[j]
                if character == '-':
                    end = regex[j + 1]
                    start = regex[j - 1]
                    range_list = ''
                    for k in range(int(ord(end) - ord(start))):
                        range_list = range_list + '|'
                        range_list = range_list + chr(ord(start) + k + 1)
                    regex = regex[0: j] + range_list + regex[j + 2:]
                    break
        print("postfix2: ", regex)

        # We use . to divide the regex to subRegex
        # example ab(b|c)*d+ will be converted to ab.bc|*.d+.
        dotLocations = []
        for i in range(len(regex) - 1):
            startSymbols = [')', "*","+", "*"]
            endSymbols = ["*", "+", ".", "|", ")"]
            if regex[i] in startSymbols and regex[i+1] not in endSymbols:
                dotLocations.append(i)
            elif regex[i].isalnum() and (regex[i+1].isalnum() or regex[i+1] == '('):
                dotLocations.append(i)

        for i in range(len(dotLocations)):
            regex = regex[:dotLocations[i] + 1 + i] + '.' + regex[dotLocations[i] + 1 + i:]
        print("postfix3: ", regex)

        # Applying Shunting Yard Algorithm
        for i in range(len(regex)):
            character = regex[i]
            # If ( -> push in stack
            if character == '(':
                stack.append(character)
            # If ) -> pop from stack until ( is found
            elif character == ')':
                while stack[-1] != '(':
                    postfix = postfix + stack[-1]
                    stack.pop()
                stack.pop()  # to remove ( from the stack

            # order the operators based on their precedence
            elif character in symbolPrecedence:
                while stack and symbolPrecedence.get(character, 0) <= symbolPrecedence.get(stack[-1], 0):
                    postfix = postfix + stack[-1]
                    stack.pop()
                stack.append(character)

            # If a normal character (not an symbol or parenthesis)
            else:
                postfix = postfix + character
        # pop any remaining symbols from the stack and appends them to the postfix string
        while stack:
            postfix = postfix + stack[-1]
            stack.pop()
        print("postfix5: ", regex)
        return postfix

In [37]:
class State:
    def __init__(self, name, transitions=[], parents=[], start=False, accepted=True):
        self.name = name
        self.parents = []
        self.transitions = []
        self.start = start
        self.accepted = accepted

    def transition(self, symbol, state):
        self.transitions.append((symbol, state))
        self.accepted = False
        state.parents.append(self)

    def parents(self):
        return self.parents.copy()

In [44]:
import graphviz
class NFA:
    def __init__(self, postfix=None, start=None, accept=None):
        self.start = start
        self.accept = accept
        if postfix and not start and not accept:
            obj = self.convertToNFA(postfix)
            self.start = obj.start
            self.accept = obj.accept

    def getStateByName(self, name):
        for state in self.getStates():
            if state.name == name:
                return state
        return None

    def getStatesByNames(self, names):
        names = names.split()
        result = []
        for name in names:
            for state in self.getStates():
                if state.name == name:
                    result.append(state)
        return result

    def isAccepted(self, states):
        for state in states:
            return state.accepted
        return False

    def getStates(self):
        visited = set()
        states = []
        queue = [self.start]
        visited.add(self.start)
        while queue:
            state = queue.pop(0)
            states.append(state)
            for (transition) in state.transitions:
                if transition[1] not in visited:
                    visited.add(transition[1])
                    queue.append(transition[1])

        return states

    # NEEDS MODIFICATION
    def getSymbols(self):
        """
        Returns the set of symbols that can be used to transition between NFA states.
        """
        states = self.getStates()
        symbols = set()
        for state in states:
            for symbol, next_state in state.transitions:
                if symbol != 'ϵ':
                    symbols.add(symbol)
        return list(symbols)

    def convertToNFA(self, postfix):
        stack = []
        stateCounter = 0
        for character in postfix:
            if character == '.':
                # state_1 is the old state in the stack
                # state_2 is the new state in the stack
                state_2 = stack.pop()
                state_1 = stack.pop()
                state_1.accept.transition('ϵ', state_2.start)
                stack.append(NFA(start = state_1.start, accept = state_2.accept))

            elif character == '*':
                state_1 = stack.pop()
                start = State('S' + str(stateCounter))
                accept = State('S' + str(stateCounter + 1))
                start.transition('ϵ', state_1.start)
                start.transition('ϵ', accept)
                state_1.accept.transition('ϵ', start)
                state_1.accept.transition('ϵ', accept)
                stack.append(NFA(start = start, accept = accept))
                stateCounter += 2

            elif character == '|':
                state_2 = stack.pop()
                state_1 = stack.pop()
                start = State('S' + str(stateCounter))
                accept = State('S' + str(stateCounter + 1))
                start.transition('ϵ', state_1.start)
                start.transition('ϵ', state_2.start)
                state_1.accept.transition('ϵ', accept)
                state_2.accept.transition('ϵ', accept)
                stack.append(NFA(start = start, accept = accept))
                stateCounter += 2

            elif character == '+':
                state_1 = stack.pop()
                start = State('S' + str(stateCounter))
                accept = State('S' + str(stateCounter + 1))
                start.transition('ϵ', state_1.start)
                state_1.accept.transition('ϵ', start)
                state_1.accept.transition('ϵ', accept)
                stack.append(NFA(start = start, accept = accept))
                stateCounter += 2

            elif character == '?':
                state_1 = stack.pop()
                start = State('S' + str(stateCounter))
                accept = State('S' + str(stateCounter + 1))
                start.transition('ϵ', state_1.start)
                start.transition('ϵ', accept)
                state_1.accept.transition('ϵ', accept)
                stack.append(NFA(start = start, accept = accept))
                stateCounter += 2

            else:
                start = State('S' + str(stateCounter))
                accept = State('S' + str(stateCounter + 1))
                start.transition(character, accept)
                stack.append(NFA(start = start, accept = accept))
                stateCounter += 2

        return stack.pop()

    def JSONIFY(self):
        states = {}
        for state in self.getStates():
            stateObj = {
                'isTerminatingState': state.accepted,
            }
            for character, nextState in state.transitions:
                if character not in stateObj:   # if one transition
                    stateObj[character] = nextState.name
                else:   # if multiple transitions to the same state
                    stateObj[character] += ',' + nextState.name
            states[state.name] = stateObj

        return {
            'startingState': self.start.name,
            **states,
        }

    def visualize(self, name, view=False):
        json = self.JSONIFY()
        graph = graphviz.Digraph(engine='dot')
        for state, transitions in json.items():
            if state == 'startingState':
                continue
            if transitions['isTerminatingState']:
                graph.node(state, shape='doublecircle')
            else:
                graph.node(state, shape='circle')
            for char, nextState in transitions.items():
                if char == 'isTerminatingState':
                    continue
                children = nextState.split(',')
                for child in children:
                    graph.edge(state, child, label=char)
        graph.render(name, view=view)
        return graph


In [45]:

cases = {0: 'ab(b|c)*d+', 1: ' (AB)', 2: '(A|B)', 3: '([A-Z])', 4: '(A)+', 5: '(A)*', 6: '(((AB)((A|B)*))(AB))', 7: '((((AB)|[A-Z])+)([A-Z]*))', 8: '(((((ABE)|C)|((([A-Z])S)*))+)((AB)C))',
          9: '((([a-z_])(([a-z0-9_])*))(([!?])?))', 10: '(A(((B*)|(DA))*))((CG)|(D([DEF])))', 11: '(ab', 12: '(a([b-c))', 13: '((a|b)|)', 14: '(a{3,2})'}
regex = cases[0]
# 7

# regex = input("Enter regular expression: ")
if not validateRegex(regex):
    print("non valiid")

postfix = postfixByShuntYard(regex)
print("----------------------------------------------------------------")
print("regex:", regex)
print("----------------------------------------------------------------")
print("postfix: ", postfix.getPostfixedRegex())
print("----------------------------------------------------------------")
nfa = NFA(postfix=postfix.getPostfixedRegex())
print("NFA: ", nfa.JSONIFY())
nfa.visualize(name='graphs/nfa_graph.gv', view=False)
print("----------------------------------------------------------------")
# dfa = DFA(nfa)
# print("DFA: ", dfa.toDict())
# dfa.visualize(name='output/dfa.gv', view=False)
# print("----------------------------------------------------------------")
# minDfa = MIN_DFA(dfa)
# print("Minimized DFA: ", minDfa.toDict())
# minDfa.visualize(name='output/min_dfa.gv', view=False)



postfix1:  ab(b|c)*d+
postfix2:  ab(b|c)*d+
postfix3:  a.b.(b|c)*.d+
postfix5:  a.b.(b|c)*.d+
----------------------------------------------------------------
regex: ab(b|c)*d+
----------------------------------------------------------------
postfix:  ab.bc|*.d+.
----------------------------------------------------------------
NFA:  {'startingState': 'S0', 'S0': {'isTerminatingState': False, 'a': 'S1'}, 'S1': {'isTerminatingState': False, 'ϵ': 'S2'}, 'S2': {'isTerminatingState': False, 'b': 'S3'}, 'S3': {'isTerminatingState': False, 'ϵ': 'S10'}, 'S10': {'isTerminatingState': False, 'ϵ': 'S8,S11'}, 'S8': {'isTerminatingState': False, 'ϵ': 'S4,S6'}, 'S11': {'isTerminatingState': False, 'ϵ': 'S14'}, 'S4': {'isTerminatingState': False, 'b': 'S5'}, 'S6': {'isTerminatingState': False, 'c': 'S7'}, 'S14': {'isTerminatingState': False, 'ϵ': 'S12'}, 'S5': {'isTerminatingState': False, 'ϵ': 'S9'}, 'S7': {'isTerminatingState': False, 'ϵ': 'S9'}, 'S12': {'isTerminatingState': False, 'd': 'S13'}, 'S